In [1]:
import re
import os
import io
import nltk
import requests
import numpy as np
import pandas as pd
# import data_helpers as dh
from textblob import Word
from zipfile import ZipFile
from nltk.tag import pos_tag
from nltk.corpus import reuters
from nltk.corpus import stopwords
from nltk import download as nltk_download
from text_processing import preprocess_spacy, clean_str
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.datasets.twenty_newsgroups import fetch_20newsgroups

stemmer = SnowballStemmer("english")
stop_words=stopwords.words('english')
nltk_download('reuters')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
full_path= '/home/silas/final_project/W266-final-project'

[nltk_data] Downloading package reuters to /home/silas/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /home/silas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/silas/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/silas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### load data (labeled)

#### climate data

In [8]:
def load_climate():

    # load = pd.read_json(full_path + '/Data/ieapandm.json', 'column')
    load = pd.read_json('/home/silas/final_project/ieapandm.json', 'column')
    load['label'] = 1
    data = load[['text', 'label']]
    data.columns = ['texts', 'labels']
    
    return data

#### spam

 - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [2]:
def load_spam():

    zip_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
    r = requests.get(zip_url)
    z = ZipFile(io.BytesIO(r.content))
    file = z.read('SMSSpamCollection')
    # Format Data
    text_data = file.decode()
    text_data = text_data.encode('ascii',errors='ignore')
    text_data = text_data.decode().split('\n')
    text_data = [x.split('\t') for x in text_data if len(x)>=1]
    df = pd.DataFrame(text_data)
    df[0] = pd.Categorical(df[0]).codes
    data = pd.DataFrame({'texts': df[1], 'labels': df[0]})
    
    return data

#### news20
- https://archive.ics.uci.edu/ml/datasets/Twenty+Newsgroups

In [3]:
def load_newsgroups():
    
    categories = ['alt.atheism', 'comp.graphics',
              'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
              'comp.windows.x', 'misc.forsale', 'rec.autos',
              'rec.motorcycles', 'rec.sport.baseball',
              'rec.sport.hockey', 'sci.crypt', 'sci.electronics',
              'sci.med', 'sci.space', 'soc.religion.christian',
              'talk.politics.guns', 'talk.politics.mideast',
              'talk.politics.misc', 'talk.religion.misc']
    newsgroups = fetch_20newsgroups(categories=categories)
    y_true= [newsgroups.target_names[idx] for idx in newsgroups.target]
    # y_true = newsgroups.target
    data = pd.DataFrame({'texts': newsgroups.data, 'labels':y_true})
    
    return data

#### Reuters
- https://archive.ics.uci.edu/ml/datasets/Reuters-21578+Text+Categorization+Collection
- (description: https://martin-thoma.com/nlp-reuters/)
- http://www.nltk.org/book/ch02.html

In [27]:
# http://www.vision.caltech.edu/Image_Datasets/Caltech101/
# https://martin-thoma.com/nlp-reuters/

n_classes = 90
labels = reuters.categories()


# def load_reuters(config={}):
def load_reuters():
    """
    Load the Reuters dataset.

    Returns
    -------
    Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """

    documents = reuters.fileids()
    test = [d for d in documents if d.startswith('test/')]
    train = [d for d in documents if d.startswith('training/')]

    docs = {}
     
    train_lst = [reuters.raw(doc_id) for doc_id in train]
    test_lst = [reuters.raw(doc_id) for doc_id in test]
    text = train_lst + test_lst
    
    train_labels = [reuters.categories(doc_id)[0] for doc_id in train]
    test_labels = [reuters.categories(doc_id)[0] for doc_id in test]
    labels = train_labels + test_labels
    
    data = pd.DataFrame({'texts':text, 'labels':labels})
    
    return data


#### BBC news
- http://mlg.ucd.ie/datasets/bbc.html
- http://mlg.ucd.ie/files/datasets/bbc.zip

In [14]:
def load_bbc():
    
    TRAIN_PATH = '/home/silas/final_project/W266-final-project/Input/bbc_dataset.csv'
    df = pd.read_csv(TRAIN_PATH)
    df_bbc = pd.DataFrame({'texts': df.news, 'labels': df.type})

    return df_bbc
#        print('writing csv flie ...')
#        df_bbc.to_csv('../bbc_dataset.csv', index=False)

#### Output

In [15]:
df_climate = load_climate()
df_spam = load_spam()
df_newsgroups = load_newsgroups()
df_reuters = load_reuters()
df_bbc = load_bbc()

### Pre-processing

In [18]:
def pre_processing_steps(df):
    
    """
    Tokenizes, removes stop words, stems (requires df "text" labeled)
    Args: dataframe
    Returns: dataframe with processed text
    """
    filtered_tokens=[]
    for doc in df.texts:
        token = nltk.word_tokenize(doc)
        filtered_tokens_temp=[]
        for tok in token:
            if tok not in stop_words:
                clean = clean_str(tok)
                if re.search('[a-zA-Z]', clean):
                    filtered_tokens_temp.append(stemmer.stem(clean))
        filtered_tokens.append(filtered_tokens_temp)
    df['processed_text'] = filtered_tokens    
    return df

In [21]:
def pre_processing_steps_spacy(df):
    texts = preprocess_spacy(df)
    df['processed_text'] = texts
    return df

In [22]:
pre_processing_steps_spacy(df_climate).to_pickle('/home/silas/final_project/W266-final-project/Data/' + '{}.pkl'.format('climate'))

FileNotFoundError: [Errno 2] No such file or directory: '/home/silas/final_project/W266-final-project/Data/climate.pkl'

### Process and Save

In [25]:
for df,df_name in zip([df_climate, df_spam, df_newsgroups,df_reuters,df_bbc],['climate','spam','newsgroups','reuters','bbc']):
    print("Preprocessing {} dataset...".format(df_name))
    pre_processing_steps_spacy(df).to_pickle('/home/silas/final_project/W266-final-project/Data/' + '{}.pkl'.format(df_name))

Preprocessing spam dataset...
Preprocessing newsgroups dataset...
Preprocessing reuters dataset...
Preprocessing bbc dataset...


### Load

In [26]:
df_spam = pd.read_pickle("./spam.pkl")
df_newsgroups = pd.read_pickle("./newsgroups.pkl")
df_reuters = pd.read_pickle("./reuters.pkl")
df_bbc = pd.read_pickle("./bbc.pkl")

### Train, test, split

def train_test_splitter(df,holdout=.2):    
    mask = np.random.rand(len(df)) < 1 - holdout
    train,test = df[mask],df[~mask]
    return train, test

train, test = train_test_splitter(df)

In [ ]:
# from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.feature_extraction.text import TfidfVectorizer